## Why is temporal collocation important?

Satellite observations usually have an irregular temporal sampling pattern (intervals between 6-36 hours), which is mostly controlled by the orbit of the satellite and the instrument measurement geometry. On the other hand, in-situ instruments or land surface models generally sample on regular time intervals (commonly every 1, 3, 6, 12 or 24 hours). 
In order to compute error/performance statistics (such as RMSD, bias, correlation) between the time series coming from different sources, it is required that observation pairs (or triplets, etc.) are found which (nearly) coincide in time.

A simple way to identify such pairs is by using a nearest neighbor search. First, one time series needs to be selected as temporal reference (i.e. all other time series will be matched to this reference) and second, a tolerance window (typically around 1-12 hours) has to be defined characterizing the temporal correlation of neighboring observation (i.e. observations outside of the tolerance window are no longer be considered as representative neighbors). 

## Temporal collocation in pytesmo

Pytesmo contains the function `pytesmo.temporal_matching.temporal_collocation` for nearest neighbour matching. It requires a reference index (can also be a DataFrame or a Series), a DataFrame (or Series) to be collocated, and a window.

```
collocated = temporal_collocation(reference, input_frame, window)
```

The window argument corresponds to the time intervals that are included in the nearest neighbour search in each direction, e.g. if the reference time is $t$ and the window $\Delta$, the nearest neighbour inside $[t-\Delta, t+\Delta]$ is returned. If no neighbour is found `np.nan` is used as replacement. NaNs can be dropped from the returned dataframe by providing the optional keyword argument ``dropna=True`` to the function.

Below are two simple examples which demonstrate the usage. The first example assumes that the index of data to be collocated is shifted by 3 hours with respect to the reference, while using a 6 hour window. The second example uses an index that is randomly shifted by $\pm12$ hours with respect to the reference. The second example also uses a 6 hour window, which results in some missing values in the resulting dataframe.

In [1]:
import numpy as np
import pandas as pd

from pytesmo.temporal_matching import temporal_collocation

# use 6 hour window for the examples
window = pd.Timedelta(hours=6)

# create reference time series
ref = pd.date_range("2020-01-01", "2020-12-31", freq="D")
# temporal_collocation can also take a DataFrame or Series as reference input,
# in case their index is a DatetimeIndex.


# create other time series as dataframe
values = np.random.randn(len(ref), 3)
shifted = pd.DataFrame(values, index=ref + pd.Timedelta(hours=3))
random_shift = np.random.uniform(-12, 12, len(ref))
random = pd.DataFrame(values, index=ref + pd.to_timedelta(random_shift, "H"))

print(shifted)
print(random)

                            0         1         2
2020-01-01 03:00:00  1.201342 -0.800769  0.401427
2020-01-02 03:00:00 -1.069162 -0.881217  0.778702
2020-01-03 03:00:00 -0.139476  0.533678  0.563714
2020-01-04 03:00:00 -0.697052 -2.088295  0.944217
2020-01-05 03:00:00  0.077201 -0.079829 -0.236811
...                       ...       ...       ...
2020-12-27 03:00:00 -0.888301 -0.851369 -1.850475
2020-12-28 03:00:00 -0.604062  0.294393 -0.566648
2020-12-29 03:00:00 -0.122994 -0.372023 -0.619184
2020-12-30 03:00:00 -1.486709  0.853604 -0.373724
2020-12-31 03:00:00 -2.053052 -0.833403  0.547443

[366 rows x 3 columns]
                                      0         1         2
2020-01-01 10:10:05.882844000  1.201342 -0.800769  0.401427
2020-01-02 02:55:57.469022400 -1.069162 -0.881217  0.778702
2020-01-03 11:07:48.032410800 -0.139476  0.533678  0.563714
2020-01-04 07:05:18.389209200 -0.697052 -2.088295  0.944217
2020-01-04 15:08:30.166195200  0.077201 -0.079829 -0.236811
...             

In [2]:
# match the regularly shifted data
matched_shifted = temporal_collocation(ref, shifted, window)
print(matched_shifted)

# the data should be the same before and after matching
assert np.all(shifted.values == matched_shifted.values)

                                  0         1         2
2020-01-01 00:00:00+00:00  1.201342 -0.800769  0.401427
2020-01-02 00:00:00+00:00 -1.069162 -0.881217  0.778702
2020-01-03 00:00:00+00:00 -0.139476  0.533678  0.563714
2020-01-04 00:00:00+00:00 -0.697052 -2.088295  0.944217
2020-01-05 00:00:00+00:00  0.077201 -0.079829 -0.236811
...                             ...       ...       ...
2020-12-27 00:00:00+00:00 -0.888301 -0.851369 -1.850475
2020-12-28 00:00:00+00:00 -0.604062  0.294393 -0.566648
2020-12-29 00:00:00+00:00 -0.122994 -0.372023 -0.619184
2020-12-30 00:00:00+00:00 -1.486709  0.853604 -0.373724
2020-12-31 00:00:00+00:00 -2.053052 -0.833403  0.547443

[366 rows x 3 columns]


In [3]:
# match the randomly shifted data
matched_random = temporal_collocation(ref, random, window)
print(matched_random)

# the data should be the same as before matching at the locations where the shift
# was below 6 hours, and should be np.nan when shift was larger
should_be_nan = np.abs(random_shift) > 6
assert np.all(matched_random[~should_be_nan].values == random[~should_be_nan].values)
assert np.all(np.isnan(matched_random[should_be_nan].values))

                                  0         1         2
2020-01-01 00:00:00+00:00       NaN       NaN       NaN
2020-01-02 00:00:00+00:00 -1.069162 -0.881217  0.778702
2020-01-03 00:00:00+00:00       NaN       NaN       NaN
2020-01-04 00:00:00+00:00       NaN       NaN       NaN
2020-01-05 00:00:00+00:00       NaN       NaN       NaN
...                             ...       ...       ...
2020-12-27 00:00:00+00:00 -0.888301 -0.851369 -1.850475
2020-12-28 00:00:00+00:00       NaN       NaN       NaN
2020-12-29 00:00:00+00:00       NaN       NaN       NaN
2020-12-30 00:00:00+00:00       NaN       NaN       NaN
2020-12-31 00:00:00+00:00 -2.053052 -0.833403  0.547443

[366 rows x 3 columns]


`temporal_collocation` can also return the original index of the data that was matched as a separate column in the resulting DataFrame, if required, and can additionally also calculate the distance to the reference. The column names are "index_other" and "distance_other", respectively.

In [4]:
# also return original index and distance
matched_shifted = temporal_collocation(ref, shifted, window, return_index=True, return_distance=True)
print(matched_shifted)

# the index should be the same as unmatched, and the distance should be 3  hours
assert np.all(matched_shifted["index_other"].values == shifted.index.values)
assert np.all(matched_shifted["distance_other"] == pd.Timedelta(hours=3))

                                  0         1         2  \
2020-01-01 00:00:00+00:00  1.201342 -0.800769  0.401427   
2020-01-02 00:00:00+00:00 -1.069162 -0.881217  0.778702   
2020-01-03 00:00:00+00:00 -0.139476  0.533678  0.563714   
2020-01-04 00:00:00+00:00 -0.697052 -2.088295  0.944217   
2020-01-05 00:00:00+00:00  0.077201 -0.079829 -0.236811   
...                             ...       ...       ...   
2020-12-27 00:00:00+00:00 -0.888301 -0.851369 -1.850475   
2020-12-28 00:00:00+00:00 -0.604062  0.294393 -0.566648   
2020-12-29 00:00:00+00:00 -0.122994 -0.372023 -0.619184   
2020-12-30 00:00:00+00:00 -1.486709  0.853604 -0.373724   
2020-12-31 00:00:00+00:00 -2.053052 -0.833403  0.547443   

                                        index_other distance_other  
2020-01-01 00:00:00+00:00 2020-01-01 03:00:00+00:00       03:00:00  
2020-01-02 00:00:00+00:00 2020-01-02 03:00:00+00:00       03:00:00  
2020-01-03 00:00:00+00:00 2020-01-03 03:00:00+00:00       03:00:00  
2020-01-04 00:0

## Flags

Satellite data often contains flags indicating quality issues with the data. With `temporal_collocation` it is possible to use this information. Flags can either be provided as array (of the same length as the input DataFrame), or the name of a column in the DataFrame to be used as flag can be provided as string. Any non-zero flag is interpreted as indicating invalid data. By default this will not be used, but when passing ``use_invalid=True``, the invalid values will be used in case no valid match was found.

For the following example, we reuse the input data shifted by 3 hours, but we will now assume that the first 3 observations had quality issues.

In [5]:
# flag the first 3 observations as invalid
flag = np.zeros(len(ref), dtype=bool)
flag[0:3] = True
flag[0:10]

array([ True,  True,  True, False, False, False, False, False, False,
       False])

In [6]:
matched_flagged = temporal_collocation(ref, shifted, window, flag=flag)
print(matched_flagged)

# the first 3 values should be NaN, otherwise the result should be the same as matched_shifted
assert np.all(np.isnan(matched_flagged.values[0:3, :]))
assert np.all(matched_flagged.values[3:, :] == matched_shifted.values[3:, 0:3])  # excluding additonal columns

                                  0         1         2
2020-01-01 00:00:00+00:00       NaN       NaN       NaN
2020-01-02 00:00:00+00:00       NaN       NaN       NaN
2020-01-03 00:00:00+00:00       NaN       NaN       NaN
2020-01-04 00:00:00+00:00 -0.697052 -2.088295  0.944217
2020-01-05 00:00:00+00:00  0.077201 -0.079829 -0.236811
...                             ...       ...       ...
2020-12-27 00:00:00+00:00 -0.888301 -0.851369 -1.850475
2020-12-28 00:00:00+00:00 -0.604062  0.294393 -0.566648
2020-12-29 00:00:00+00:00 -0.122994 -0.372023 -0.619184
2020-12-30 00:00:00+00:00 -1.486709  0.853604 -0.373724
2020-12-31 00:00:00+00:00 -2.053052 -0.833403  0.547443

[366 rows x 3 columns]


In [7]:
# This also works when the flag is already in the input frame, but note that
# in the output frame the nonzero flag values are replaced by NaN
flagged = shifted.assign(my_flag=flag)
matched_flagged = temporal_collocation(ref, flagged, window, flag="my_flag")
print(matched_flagged)

# the first 3 values should be NaN, otherwise the result should be the same as matched_shifted
assert np.all(np.isnan(matched_flagged.iloc[0:3, 0:3].values))
assert np.all(matched_flagged.iloc[3:, 0:3].values == matched_shifted.values[3:, 0:3])  # excluding additonal columns

                                  0         1         2 my_flag
2020-01-01 00:00:00+00:00       NaN       NaN       NaN     NaN
2020-01-02 00:00:00+00:00       NaN       NaN       NaN     NaN
2020-01-03 00:00:00+00:00       NaN       NaN       NaN     NaN
2020-01-04 00:00:00+00:00 -0.697052 -2.088295  0.944217   False
2020-01-05 00:00:00+00:00  0.077201 -0.079829 -0.236811   False
...                             ...       ...       ...     ...
2020-12-27 00:00:00+00:00 -0.888301 -0.851369 -1.850475   False
2020-12-28 00:00:00+00:00 -0.604062  0.294393 -0.566648   False
2020-12-29 00:00:00+00:00 -0.122994 -0.372023 -0.619184   False
2020-12-30 00:00:00+00:00 -1.486709  0.853604 -0.373724   False
2020-12-31 00:00:00+00:00 -2.053052 -0.833403  0.547443   False

[366 rows x 4 columns]


## Handling of timezones

`temporal_collocation` can also handle timezones. For this example, we use as the reference index from above, but shifted by 8 hours (to 8 am each day), and set it to UTC. The other DataFrame index is in Pacific time and remains at midnight. Since Pacific time is at -8 compared to UTC, they both refer to the same times (at least in winter, without DST).


Note that if no timezone information was given in the reference, it is assumed to be in UTC, and the resulting DataFrame will also be in UTC. When ``return_index=True``, the index of the input frame that is returned has been converted to the same time zone as the reference.

In [8]:
# shift by +8 hours
utc_shifted = ref.copy().tz_localize("UTC") + pd.Timedelta(hours=8)
utc_shifted

DatetimeIndex(['2020-01-01 08:00:00+00:00', '2020-01-02 08:00:00+00:00',
               '2020-01-03 08:00:00+00:00', '2020-01-04 08:00:00+00:00',
               '2020-01-05 08:00:00+00:00', '2020-01-06 08:00:00+00:00',
               '2020-01-07 08:00:00+00:00', '2020-01-08 08:00:00+00:00',
               '2020-01-09 08:00:00+00:00', '2020-01-10 08:00:00+00:00',
               ...
               '2020-12-22 08:00:00+00:00', '2020-12-23 08:00:00+00:00',
               '2020-12-24 08:00:00+00:00', '2020-12-25 08:00:00+00:00',
               '2020-12-26 08:00:00+00:00', '2020-12-27 08:00:00+00:00',
               '2020-12-28 08:00:00+00:00', '2020-12-29 08:00:00+00:00',
               '2020-12-30 08:00:00+00:00', '2020-12-31 08:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=366, freq='D')

In [9]:
# in pacific time it's now midnight
pacific_time = ref.copy().tz_localize("US/Pacific")
pacific = pd.DataFrame(values, index=pacific_time)

pacific_time

DatetimeIndex(['2020-01-01 00:00:00-08:00', '2020-01-02 00:00:00-08:00',
               '2020-01-03 00:00:00-08:00', '2020-01-04 00:00:00-08:00',
               '2020-01-05 00:00:00-08:00', '2020-01-06 00:00:00-08:00',
               '2020-01-07 00:00:00-08:00', '2020-01-08 00:00:00-08:00',
               '2020-01-09 00:00:00-08:00', '2020-01-10 00:00:00-08:00',
               ...
               '2020-12-22 00:00:00-08:00', '2020-12-23 00:00:00-08:00',
               '2020-12-24 00:00:00-08:00', '2020-12-25 00:00:00-08:00',
               '2020-12-26 00:00:00-08:00', '2020-12-27 00:00:00-08:00',
               '2020-12-28 00:00:00-08:00', '2020-12-29 00:00:00-08:00',
               '2020-12-30 00:00:00-08:00', '2020-12-31 00:00:00-08:00'],
              dtype='datetime64[ns, US/Pacific]', length=366, freq='D')

In [10]:
# but when converting the timezones to utc both, there's no time difference
pacific_time.tz_convert("UTC") - utc_shifted

TimedeltaIndex(['00:00:00', '00:00:00', '00:00:00', '00:00:00', '00:00:00',
                '00:00:00', '00:00:00', '00:00:00', '00:00:00', '00:00:00',
                ...
                '00:00:00', '00:00:00', '00:00:00', '00:00:00', '00:00:00',
                '00:00:00', '00:00:00', '00:00:00', '00:00:00', '00:00:00'],
               dtype='timedelta64[ns]', length=366, freq=None)

In [11]:
matched = temporal_collocation(utc_shifted, pacific, window, return_index=True)
print(matched)

# the first 50 (winter) time indices should be the same, and since all are within
# the 6 hour window, there should be no NaNs
assert np.all(matched.index[0:50] == matched["index_other"][0:50])
assert not np.any(np.isnan(matched.iloc[:, 0:3]))

                                  0         1         2  \
2020-01-01 08:00:00+00:00  1.201342 -0.800769  0.401427   
2020-01-02 08:00:00+00:00 -1.069162 -0.881217  0.778702   
2020-01-03 08:00:00+00:00 -0.139476  0.533678  0.563714   
2020-01-04 08:00:00+00:00 -0.697052 -2.088295  0.944217   
2020-01-05 08:00:00+00:00  0.077201 -0.079829 -0.236811   
...                             ...       ...       ...   
2020-12-27 08:00:00+00:00 -0.888301 -0.851369 -1.850475   
2020-12-28 08:00:00+00:00 -0.604062  0.294393 -0.566648   
2020-12-29 08:00:00+00:00 -0.122994 -0.372023 -0.619184   
2020-12-30 08:00:00+00:00 -1.486709  0.853604 -0.373724   
2020-12-31 08:00:00+00:00 -2.053052 -0.833403  0.547443   

                                        index_other  
2020-01-01 08:00:00+00:00 2020-01-01 08:00:00+00:00  
2020-01-02 08:00:00+00:00 2020-01-02 08:00:00+00:00  
2020-01-03 08:00:00+00:00 2020-01-03 08:00:00+00:00  
2020-01-04 08:00:00+00:00 2020-01-04 08:00:00+00:00  
2020-01-05 08:00:00+0